In [1]:
import time
import clr
import os

In [2]:
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.KCube.DCServoCLI.dll")

clr.AddReference(r"C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.TCube.DCServoCLI.dll")

from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import KCubeMotor
from Thorlabs.MotionControl.GenericMotorCLI.ControlParameters import JogParametersBase
from Thorlabs.MotionControl.KCube.DCServoCLI import *
from System import Decimal

from Thorlabs.MotionControl.TCube.DCServoCLI import *

In [3]:
# Device SN
serial_num = str('83815214')

In [ ]:
DeviceManagerCLI.BuildDeviceList()
controller = TCubeDCServo.CreateTCubeDCServo(serial_num)
# controller = KCubeDCServo.CreateKCubeDCServo(serial_num)

if not controller == None:
    controller.Connect(serial_num)
    
    if not controller.IsSettingsInitialized():
        controller.WaitForSettingsInitialized(3000)
        
    controller.StartPolling(50)
    time.sleep(.1)
    controller.EnableDevice()
    time.sleep(.1)
    
    config = controller.LoadMotorConfiguration(serial_num, DeviceConfiguration.DeviceSettingsUseOptionType.UseFileSettings)
    config.DeviceSettingsName = str('Z725B')
    config.UpdateCurrentConfiguration()
    controller.SetSettings(controller.MotorDeviceSettings, True, False)
    
    print('Homing Motor')
    #Hacemos que el tornillo se mueva hasta su posición base
    controller.Home(600000)
    
    jog_params = controller.GetJogParams()
    jog_params.StepSize = Decimal(.5) #Cambiamos cuanto se mueve nuestro tornillo en cada paso 
    jog_params.MaxVelocity = Decimal(1) #Cambiamos la velocidad máxima a la que se mueve en cada paso
    jog_params.JogMode = JogParametersBase.JogModes.SingleStep 
    
    controller.SetJogParams(jog_params) #Incorporamos los nuevos parámetros a las instrucciones de movimiento
    
    pause_time = 5 #Tiempo de pausa entre movimientos (en segundos)
    forward_movements = 3 #Number of forward movements
    backward_movements = 0 #Number of backward movements
    start_time = time.time() #Iniciamos el origen de tiempos

    for _ in range(forward_movements):
        print('Moving Motor Forward at t = ', time.time() - start_time)
        controller.MoveJog(MotorDirection.Forward, 60000)
        print(f'Position: {controller.Position}, time: {time.time() - start_time}')
        time.sleep(pause_time)
        print(f'End pause at time :{time.time() - start_time}')

    for _ in range(backward_movements):
        print('Moving Motor Backward at t = ', time.time() - start_time)
        controller.MoveJog(MotorDirection.Backward, 60000)
        print(f'Position: {controller.Position}, time: {time.time() - start_time}')
        time.sleep(pause_time)
        print(f'End pause at time :{time.time() - start_time}')

    
    print('FINISHED at t = ', time.time() - start_time)
    controller.StopPolling()
    controller.Disconnect(False)

Homing Motor
Moving Motor Forward at t =  0.0
Position: 0.50007, time: 2.3732757568359375
End pause at time :7.374685049057007
Moving Motor Forward at t =  7.374685049057007
Position: 0.99997, time: 10.002147197723389
End pause at time :15.002880096435547
Moving Motor Forward at t =  15.002880096435547
Position: 1.5, time: 17.51856756210327
End pause at time :22.519659519195557
Moving Motor Backward at t =  22.519659519195557
Position: 0.99994, time: 24.939652919769287
End pause at time :29.941157579421997
Moving Motor Backward at t =  29.941157579421997
Position: 0.49996, time: 32.37703561782837
End pause at time :37.37851643562317
Moving Motor Backward at t =  37.37851643562317
Position: -0.00009, time: 39.66991400718689
End pause at time :44.67097592353821
FINISHED at t =  44.67097592353821
